Sweep Calib dataset size: 64, 128, 256, 512. Use the same dataset for generating Ltilde

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt


with open('./Ltilde/cached_grad_bert_calib_w-8-4-2_calib64.pkl','rb') as f:
    hm64 = pickle.load(f)
with open('./Ltilde/cached_grad_bert_calib_w-8-4-2_calib128.pkl','rb') as f:
    hm128 = pickle.load(f)
with open('./Ltilde/cached_grad_bert_calib_w-8-4-2_calib256.pkl','rb') as f:
    hm256 = pickle.load(f)
with open('./Ltilde/cached_grad_bert_calib_w-8-4-2_calib512.pkl','rb') as f:
    hm512 = pickle.load(f)    


# use the created array to output your multiple images. In this case I have stacked 4 images vertically
print("Calib and Ltilde size: 64 samples, mean abs Ltilde: ", np.mean(np.abs(hm64['Ltilde'])),"  min: ", np.min(hm64['Ltilde']),"  max: ", np.max(hm64['Ltilde']))
print("Calib and Ltilde size: 128 samples, mean abs Ltilde: ", np.mean(np.abs(hm128['Ltilde'])),"  min: ", np.min(hm128['Ltilde']),"  max: ", np.max(hm128['Ltilde']))
print("Calib and Ltilde size: 256 samples, mean abs Ltilde: ", np.mean(np.abs(hm256['Ltilde'])),"  min: ", np.min(hm256['Ltilde']),"  max: ", np.max(hm256['Ltilde']))
print("Calib and Ltilde size: 512 samples, mean abs Ltilde: ", np.mean(np.abs(hm512['Ltilde'])),"  min: ", np.min(hm512['Ltilde']),"  max: ", np.max(hm512['Ltilde']))
plt.rcParams['figure.figsize'] = (12,3)
plt.subplot(1, 4, 1)
plt.imshow(hm64['Ltilde'], vmin=-0.1, vmax=1.7, cmap='hot', aspect='auto', interpolation='nearest')
plt.title("Calib size: 64, Ltilde size: 64", size=11)
plt.subplot(1, 4, 2)
plt.imshow(hm128['Ltilde'], vmin=-0.1, vmax=1.7, cmap='hot', aspect='auto', interpolation='nearest')
plt.title("Calib size: 128, Ltilde size: 128", size=11)
plt.subplot(1, 4, 3)
plt.imshow(hm256['Ltilde'], vmin=-0.1, vmax=1.7, cmap='hot', aspect='auto', interpolation='nearest')
plt.title("Calib size: 256, Ltilde size: 256", size=11)
plt.subplot(1, 4, 4)
plt.imshow(hm512['Ltilde'], vmin=-0.1, vmax=1.7, cmap='hot', aspect='auto', interpolation='nearest')
plt.title("Calib size: 512, Ltilde size: 512", size=11)
plt.colorbar()
plt.tight_layout()
plt.show()    


Fix the calibrated model using 128 samples,  Use subset of the same dataset for computing Ltilde

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

with open('./Ltilde/cached_grad_bert_calib_w-8-4-2_calib128_subset16.pkl','rb') as f:
    hm128_16 = pickle.load(f)
with open('./Ltilde/cached_grad_bert_calib_w-8-4-2_calib128_subset32.pkl','rb') as f:
    hm128_32 = pickle.load(f)
with open('./Ltilde/cached_grad_bert_calib_w-8-4-2_calib128_subset64.pkl','rb') as f:
    hm128_64 = pickle.load(f)
with open('./Ltilde/cached_grad_bert_calib_w-8-4-2_calib128.pkl','rb') as f:
    hm128_128 = pickle.load(f)    


# use the created array to output your multiple images. In this case I have stacked 4 images vertically
print("Calib size 128, Ltilde samples 16, mean abs Ltilde: ", np.mean(np.abs(hm128_16['Ltilde'])),"  min: ", np.min(hm128_16['Ltilde']),"  max: ", np.max(hm128_16['Ltilde']))
print("Calib size 128, Ltilde samples 32, mean abs Ltilde: ", np.mean(np.abs(hm128_32['Ltilde'])),"  min: ", np.min(hm128_32['Ltilde']),"  max: ", np.max(hm128_32['Ltilde']))
print("Calib size 128, Ltilde samples 64, mean abs Ltilde: ", np.mean(np.abs(hm128_64['Ltilde'])),"  min: ", np.min(hm128_64['Ltilde']),"  max: ", np.max(hm128_64['Ltilde']))
print("Calib size 128, Ltilde samples 128, mean abs Ltilde: ", np.mean(np.abs(hm128_128['Ltilde'])),"  min: ", np.min(hm128_128['Ltilde']),"  max: ", np.max(hm128_128['Ltilde']))
plt.rcParams['figure.figsize'] = (12,3)
plt.subplot(1, 4, 1)
plt.imshow(hm128_128['Ltilde'], vmin=-0.1, vmax=1.7, cmap='hot', aspect='auto', interpolation='nearest')
plt.title("Calib size: 128, Ltilde size: 128", size=11)
plt.subplot(1, 4, 2)
plt.imshow(hm128_64['Ltilde'], vmin=-0.1, vmax=1.7, cmap='hot', aspect='auto', interpolation='nearest')
plt.title("Calib size: 128, Ltilde size: 64", size=11)
plt.subplot(1, 4, 3)
plt.imshow(hm128_32['Ltilde'], vmin=-0.1, vmax=1.7, cmap='hot', aspect='auto', interpolation='nearest')
plt.title("Calib size: 128, Ltilde size: 32", size=11)
plt.subplot(1, 4, 4)
plt.imshow(hm128_16['Ltilde'], vmin=-0.1, vmax=1.7, cmap='hot', aspect='auto', interpolation='nearest')
plt.title("Calib size: 128, Ltilde size: 16", size=11)
plt.colorbar()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pickle

with open('./Clado_Naive_res/clado_naive_a8_w8-4-2_calib64.pkl','rb') as f:
    res64 = pickle.load(f)
with open('./Clado_Naive_res/clado_naive_a8_w8-4-2_calib128.pkl','rb') as f:
    res128 = pickle.load(f)
with open('./Clado_Naive_res/clado_naive_a8_w8-4-2_calib128_subset64.pkl','rb') as f:
    res128_64 = pickle.load(f)
with open('./Clado_Naive_res/clado_naive_a8_w8-4-2_calib128_subset32.pkl','rb') as f:
    res128_32 = pickle.load(f)
with open('./Clado_Naive_res/clado_naive_a8_w8-4-2_calib128_subset16.pkl','rb') as f:
    res128_16 = pickle.load(f)            
with open('./Clado_Naive_res/clado_naive_a8_w8-4-2_calib256.pkl','rb') as f:
    res256 = pickle.load(f)
with open('./Clado_Naive_res/clado_naive_a8_w8-4-2_calib512.pkl','rb') as f:
    res512 = pickle.load(f)
clado_eval_f1_64, clado_eval_exact_match_64, clado_size_64, clado_bitops_64 = [], [], [], []
naive_eval_f1_64, naive_eval_exact_match_64, naive_size_64, naive_bitops_64 = [], [], [], []
clado_eval_f1_128, clado_eval_exact_match_128, clado_size_128, clado_bitops_128 = [], [], [], []
naive_eval_f1_128, naive_eval_exact_match_128, naive_size_128, naive_bitops_128 = [], [], [], []
clado_eval_f1_256, clado_eval_exact_match_256, clado_size_256, clado_bitops_256 = [], [], [], []
naive_eval_f1_256, naive_eval_exact_match_256, naive_size_256, naive_bitops_256 = [], [], [], []
clado_eval_f1_512, clado_eval_exact_match_512, clado_size_512, clado_bitops_512 = [], [], [], []
naive_eval_f1_512, naive_eval_exact_match_512, naive_size_512, naive_bitops_512 = [], [], [], []
clado_eval_f1_128_64, clado_eval_exact_match_128_64, clado_size_128_64, clado_bitops_128_64 = [], [], [], []
naive_eval_f1_128_64, naive_eval_exact_match_128_64, naive_size_128_64, naive_bitops_128_64 = [], [], [], []
clado_eval_f1_128_32, clado_eval_exact_match_128_32, clado_size_128_32, clado_bitops_128_32 = [], [], [], []
naive_eval_f1_128_32, naive_eval_exact_match_128_32, naive_size_128_32, naive_bitops_128_32 = [], [], [], []
clado_eval_f1_128_16, clado_eval_exact_match_128_16, clado_size_128_16, clado_bitops_128_16 = [], [], [], []
naive_eval_f1_128_16, naive_eval_exact_match_128_16, naive_size_128_16, naive_bitops_128_16 = [], [], [], []



In [ ]:
for item in res64['clado_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    clado_eval_f1_64.append(eval_f1)
    clado_eval_exact_match_64.append(eval_exact_match)
    clado_size_64.append(size)
    clado_bitops_64.append(bitops)

for item in res64['naive_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    naive_eval_f1_64.append(eval_f1)
    naive_eval_exact_match_64.append(eval_exact_match)
    naive_size_64.append(size)
    naive_bitops_64.append(bitops)
    
for item in res128['clado_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    clado_eval_f1_128.append(eval_f1)
    clado_eval_exact_match_128.append(eval_exact_match)
    clado_size_128.append(size)
    clado_bitops_128.append(bitops)

for item in res128['naive_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    naive_eval_f1_128.append(eval_f1)
    naive_eval_exact_match_128.append(eval_exact_match)
    naive_size_128.append(size)
    naive_bitops_128.append(bitops)

for item in res256['clado_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    clado_eval_f1_256.append(eval_f1)
    clado_eval_exact_match_256.append(eval_exact_match)
    clado_size_256.append(size)
    clado_bitops_256.append(bitops)

for item in res256['naive_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    naive_eval_f1_256.append(eval_f1)
    naive_eval_exact_match_256.append(eval_exact_match)
    naive_size_256.append(size)
    naive_bitops_256.append(bitops)

for item in res512['clado_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    clado_eval_f1_512.append(eval_f1)
    clado_eval_exact_match_512.append(eval_exact_match)
    clado_size_512.append(size)
    clado_bitops_512.append(bitops)

for item in res512['naive_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    naive_eval_f1_512.append(eval_f1)
    naive_eval_exact_match_512.append(eval_exact_match)
    naive_size_512.append(size)
    naive_bitops_512.append(bitops)

for item in res128_64['clado_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    clado_eval_f1_128_64.append(eval_f1)
    clado_eval_exact_match_128_64.append(eval_exact_match)
    clado_size_128_64.append(size)
    clado_bitops_128_64.append(bitops)

for item in res128_64['naive_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    naive_eval_f1_128_64.append(eval_f1)
    naive_eval_exact_match_128_64.append(eval_exact_match)
    naive_size_128_64.append(size)
    naive_bitops_128_64.append(bitops)

for item in res128_32['clado_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    clado_eval_f1_128_32.append(eval_f1)
    clado_eval_exact_match_128_32.append(eval_exact_match)
    clado_size_128_32.append(size)
    clado_bitops_128_32.append(bitops)

for item in res128_32['naive_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    naive_eval_f1_128_32.append(eval_f1)
    naive_eval_exact_match_128_32.append(eval_exact_match)
    naive_size_128_32.append(size)
    naive_bitops_128_32.append(bitops)

for item in res128_16['clado_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    clado_eval_f1_128_16.append(eval_f1)
    clado_eval_exact_match_128_16.append(eval_exact_match)
    clado_size_128_16.append(size)
    clado_bitops_128_16.append(bitops)

for item in res128_16['naive_res']:
    eval_f1, eval_exact_match, _ , size, bitops, _ = item
    naive_eval_f1_128_16.append(eval_f1)
    naive_eval_exact_match_128_16.append(eval_exact_match)
    naive_size_128_16.append(size)
    naive_bitops_128_16.append(bitops)

F1 score: Sweep Calib dataset size: 64, 128, 256, 512. Use the same dataset for generating Ltilde

In [ ]:
plt.rcParams['figure.figsize'] = (12,8)
plt.axis([0.05, 0.40, 0, 50])
plt.title('Bert-tiny eval on SQuAD v1')

plt.subplot(2, 2, 1)
plt.plot(clado_size_64,clado_eval_f1_64,label='Cross-layer dependency aware',marker='v')
plt.plot(naive_size_64,naive_eval_f1_64,label='Cross-layer dependency unaware',marker='^')
plt.axhline(y=48.7715, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 64; Clado, Naive size: 64')
plt.ylabel('F1 Score')
plt.xlabel('Model Size (MB)')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(clado_size_128,clado_eval_f1_128,label='Cross-layer dependency aware',marker='v')
plt.plot(naive_size_128,naive_eval_f1_128,label='Cross-layer dependency unaware',marker='^')
plt.axhline(y=48.9000, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 128; Clado, Naive size: 128')
plt.ylabel('F1 Score')
plt.xlabel('Model Size (MB)')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(clado_size_256,clado_eval_f1_256,label='Cross-layer dependency aware',marker='v')
plt.plot(naive_size_256,naive_eval_f1_256,label='Cross-layer dependency unaware',marker='^')
plt.axhline(y=48.7525, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 256; Clado, Naive size: 256')
plt.ylabel('F1 Score')
plt.xlabel('Model Size (MB)')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(clado_size_512,clado_eval_f1_512,label='Cross-layer dependency aware',marker='v')
plt.plot(naive_size_512,naive_eval_f1_512,label='Cross-layer dependency unaware',marker='^')
plt.axhline(y=48.8238, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 512; Clado, Naive size: 512')
plt.ylabel('F1 Score')
plt.xlabel('Model Size (MB)')
plt.legend()
plt.tight_layout()
plt.show()

Exact Match Score: Sweep Calib dataset size: 64, 128, 256, 512. Use the same dataset for generating Ltilde

In [ ]:
plt.rcParams['figure.figsize'] = (12,8)
plt.axis([0.05, 0.40, 0, 50])
plt.title('Bert-tiny eval on SQuAD v1')

plt.subplot(2, 2, 1)
plt.plot(clado_size_64,clado_eval_exact_match_64,label='Cross-layer dependency aware',marker='o', color='g')
plt.plot(naive_size_64,naive_eval_exact_match_64,label='Cross-layer dependency unaware',marker='*', color='r')
plt.axhline(y=37.0199, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 64; Clado, Naive size: 64')
plt.ylabel('Exact Match Score')
plt.xlabel('Model Size (MB)')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(clado_size_128,clado_eval_exact_match_128,label='Cross-layer dependency aware',marker='o', color='g')
plt.plot(naive_size_128,naive_eval_exact_match_128,label='Cross-layer dependency unaware',marker='*', color='r')
plt.axhline(y=37.0955, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 128; Clado, Naive size: 128')
plt.ylabel('Exact Match Score')
plt.xlabel('Model Size (MB)')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(clado_size_256,clado_eval_exact_match_256,label='Cross-layer dependency aware',marker='o', color='g')
plt.plot(naive_size_256,naive_eval_exact_match_256,label='Cross-layer dependency unaware',marker='*', color='r')
plt.axhline(y=36.9158, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 256; Clado, Naive size: 256')
plt.ylabel('Exact Match Score')
plt.xlabel('Model Size (MB)')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(clado_size_512,clado_eval_exact_match_512,label='Cross-layer dependency aware',marker='o', color='g')
plt.plot(naive_size_512,naive_eval_exact_match_512,label='Cross-layer dependency unaware',marker='*', color='r')
plt.axhline(y=37.0672, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 512; Clado, Naive size: 512')
plt.ylabel('Exact Match Score')
plt.xlabel('Model Size (MB)')
plt.legend()
plt.tight_layout()
plt.show()


F1 Score: Fix the calibrated model using 128 samples,  Use subset of the same dataset for computing Ltilde

In [ ]:
plt.rcParams['figure.figsize'] = (12,8)
plt.axis([0.05, 0.40, 0, 50])
plt.title('Bert-tiny eval on SQuAD v1')
plt.subplot(2, 2, 1)
plt.plot(clado_size_128,clado_eval_f1_128,label='Cross-layer dependency aware',marker='o')
plt.plot(naive_size_128,naive_eval_f1_128,label='Cross-layer dependency unaware',marker='*')
plt.axhline(y=48.9000, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 128; Clado, Naive size: 128')
plt.ylabel('F1 Score')
plt.xlabel('Model Size (MB)')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(clado_size_128_64,clado_eval_f1_128_64,label='Cross-layer dependency aware',marker='o')
plt.plot(naive_size_128_64,naive_eval_f1_128_64,label='Cross-layer dependency unaware',marker='*')
plt.axhline(y=48.9000, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 128; Clado, Naive size: 64')
plt.ylabel('F1 Score')
plt.xlabel('Model Size (MB)')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(clado_size_128_32,clado_eval_f1_128_32,label='Cross-layer dependency aware',marker='o')
plt.plot(naive_size_128_32,naive_eval_f1_128_32,label='Cross-layer dependency unaware',marker='*')
plt.axhline(y=48.9000, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 128; Clado, Naive size: 32')
plt.ylabel('F1 Score')
plt.xlabel('Model Size (MB)')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(clado_size_128_16,clado_eval_f1_128_16,label='Cross-layer dependency aware',marker='o')
plt.plot(naive_size_128_16,naive_eval_f1_128_16,label='Cross-layer dependency unaware',marker='*')
plt.axhline(y=48.9000, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 128; Clado, Naive size: 16')
plt.ylabel('F1 Score')
plt.xlabel('Model Size (MB)')
plt.legend()
plt.tight_layout()
plt.show()


Exact Match Score: Fix the calibrated model using 128 samples,  Use subset of the same dataset for computing Ltilde

In [ ]:
plt.rcParams['figure.figsize'] = (12,8)
plt.axis([0.05, 0.40, 0, 50])
plt.title('Bert-tiny eval on SQuAD v1')

plt.subplot(2, 2, 1)
plt.plot(clado_size_128,clado_eval_exact_match_128,label='Cross-layer dependency aware',marker='o', color='g')
plt.plot(naive_size_128,naive_eval_exact_match_128,label='Cross-layer dependency unaware',marker='*', color='r')
plt.axhline(y=37.0199, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 128; Clado, Naive size: 128')
plt.ylabel('Exact Match Score')
plt.xlabel('Model Size (MB)')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(clado_size_128_64,clado_eval_exact_match_128_64,label='Cross-layer dependency aware',marker='o', color='g')
plt.plot(naive_size_128_64,naive_eval_exact_match_128_64,label='Cross-layer dependency unaware',marker='*', color='r')
plt.axhline(y=37.0955, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 128; Clado, Naive size: 64')
plt.ylabel('Exact Match Score')
plt.xlabel('Model Size (MB)')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(clado_size_128_32,clado_eval_exact_match_128_32,label='Cross-layer dependency aware',marker='o', color='g')
plt.plot(naive_size_128_32,naive_eval_exact_match_128_32,label='Cross-layer dependency unaware',marker='*', color='r')
plt.axhline(y=36.9158, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 128; Clado, Naive size: 32')
plt.ylabel('Exact Match Score')
plt.xlabel('Model Size (MB)')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(clado_size_128_16,clado_eval_exact_match_128_16,label='Cross-layer dependency aware',marker='o', color='g')
plt.plot(naive_size_128_16,naive_eval_exact_match_128_16,label='Cross-layer dependency unaware',marker='*', color='r')
plt.axhline(y=37.0672, color='gray', linestyle='--', label='Baseline: (8, 8)')
plt.title('Calib size: 128; Clado, Naive size: 16')
plt.ylabel('Exact Match Score')
plt.xlabel('Model Size (MB)')
plt.legend()
plt.tight_layout()
plt.show()
